In [1]:
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)
from coinFunc import *

In [2]:
import warnings
warnings.filterwarnings('ignore')

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Section 1: BTC, Data from Exchanges

The method "get_quandl_data()" returns a dataset of bitcoin price from a given exchange. Below this is demonstrated for the KRAKEN exchange. The dataset contains columns for opening price, daily highs and lows, and other quantitites; the currency is in dollars. Let's do a plot of the price (in dollars) of BTC according to the KRAKEN exchange.

In [3]:
btc_usd_price_kraken = get_quandl_data('BCHARTS/KRAKENUSD')
btc_usd_price_kraken.head()
btc_trace = go.Scatter(x=btc_usd_price_kraken.index, y=btc_usd_price_kraken['Weighted Price'])
py.iplot([btc_trace])

Loaded BCHARTS/KRAKENUSD from cache


Now we will download data-frames from several exchanges and save them all in a list, "exchange_data".

In [4]:
exchanges = ['KRAKEN','COINBASE','BITSTAMP','ITBIT']

exchange_data = {}

#exchange_data['KRAKEN'] = btc_usd_price_kraken

for exchange in exchanges:
    exchange_code = 'BCHARTS/{}USD'.format(exchange)
    btc_exchange_df = get_quandl_data(exchange_code)
    exchange_data[exchange] = btc_exchange_df

Loaded BCHARTS/KRAKENUSD from cache
Loaded BCHARTS/COINBASEUSD from cache
Loaded BCHARTS/BITSTAMPUSD from cache
Loaded BCHARTS/ITBITUSD from cache


From "exchange data", we will select specifically the "weighted price" column. Now we can compare "weighted price" across the various exchanges.

In [5]:
btc_usd_datasets = merge_dfs_on_column(list(exchange_data.values()), list(exchange_data.keys()), 'Weighted Price')
btc_usd_datasets.replace(0, np.nan, inplace=True)
df_scatter(btc_usd_datasets, 'Bitcoin Price (USD) By Exchange')

In [6]:
btc_usd_datasets['avg_btc_price_usd'] = btc_usd_datasets.mean(axis=1)
btc_trace = go.Scatter(x=btc_usd_datasets.index, y=btc_usd_datasets['avg_btc_price_usd'])
py.iplot([btc_trace])

## Section 2: Altcoins

In [7]:
altcoins = ['ETH','LTC','XMR','DASH','ZEC']

altcoin_data = {}
for altcoin in altcoins:
    coinpair = 'BTC_{}'.format(altcoin)
    crypto_price_df = get_crypto_data(coinpair)
    altcoin_data[altcoin] = crypto_price_df
    
altcoin_data['ETH'].tail()

Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_ETH&start=1420092000.0&end=1554000337.276706&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_LTC&start=1420092000.0&end=1554000337.297087&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_XMR&start=1420092000.0&end=1554000337.324336&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_DASH&start=1420092000.0&end=1554000337.333565&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_ZEC&start=1420092000.0&end=1554000337.345699&period=86400 from cache


,close,high,low,open,quoteVolume,volume,weightedAverage
date,,,,,,,
2019-03-26,0.033950,0.034159,0.033816,0.033983,4074.808305,138.574038,0.034007
2019-03-27,0.034520,0.034634,0.033939,0.033939,16189.602720,556.220622,0.034357
2019-03-28,0.034251,0.034574,0.034100,0.034505,2798.628890,96.033657,0.034315
2019-03-29,0.034782,0.034782,0.034100,0.034251,11091.366981,384.390722,0.034657
2019-03-30,0.034690,0.035329,0.034448,0.034782,7927.394473,276.316107,0.034856


In [8]:
for altcoin in altcoin_data.keys():
    altcoin_data[altcoin]['price_usd'] =  altcoin_data[altcoin]['weightedAverage'] * btc_usd_datasets['avg_btc_price_usd']

combined_df = merge_dfs_on_column(list(altcoin_data.values()), list(altcoin_data.keys()), 'price_usd')
combined_df['BTC'] = btc_usd_datasets['avg_btc_price_usd']
df_scatter(combined_df, 'Cryptocurrency Prices (USD)', seperate_y_axis=False, y_axis_label='Coin Value (USD)', scale='log')

In [9]:
combined_df_2016 = combined_df[combined_df.index.year == 2016]
combined_df_2016.pct_change().corr(method='pearson')
heat_2016=correlation_heatmap(combined_df_2016.pct_change(), "Cryptocurrency Correlations in 2016")

In [10]:
combined_df_2017 = combined_df[combined_df.index.year == 2017]
combined_df_2017.pct_change().corr(method='pearson')
heat_2017=correlation_heatmap(combined_df_2017.pct_change(), "Cryptocurrency Correlations in 2017")

## Section 3: Exploring Correlations in Privacy Coins

First, consider the USD prices of ZEC and XMR. What is the ratio of these prices as a function of time?

In [11]:
combined_df_sub = combined_df[combined_df.index.year > 2017]
combined_df_sub['ZEC/XMR'] = combined_df_sub["ZEC"]/combined_df_sub["XMR"]
combined_df_sub.iloc[1390:1395]

,ETH,LTC,XMR,DASH,ZEC,BTC,ZEC/XMR
date,,,,,,,


In [12]:
zec_xmr_trace = go.Scatter(x=combined_df_sub.index, y=combined_df_sub['ZEC/XMR'],name="ZEC/XMR")
py.iplot([zec_xmr_trace])

Second, how does the price of DASH compare?

In [13]:
combined_df_sub['DASH/XMR'] = combined_df_sub["DASH"]/combined_df_sub["XMR"]
dash_xmr_trace = go.Scatter(x=combined_df_sub.index, y=combined_df_sub['DASH/XMR'],name = "DASH/XMR")

data = [dash_xmr_trace, zec_xmr_trace]
py.iplot(data, filename='scatter-mode')

Notice how, in autumn 2017, the ratios are comparatively flat. This was the period of time when the price of bitcoin skyrocketed. It would be interesting to study the structure of these ratios during crypto-booms.

Third, how do these ratios behave as functions of (price of XMR)?

## Candlestick Plot of ZEC

In [72]:
zec_sub = altcoin_data["ZEC"][altcoin_data["ZEC"].index.year > 2017]

INCREASING_COLOR = '#17BECF'
DECREASING_COLOR = '#7F7F7F'

data = [ dict(
    type = 'candlestick',
    open = zec_sub["open"],
    high = zec_sub["high"],
    low = zec_sub["low"],
    close = zec_sub["close"],
    x = zec_sub.index,
    yaxis = 'y2',
    name = 'GS',
    increasing = dict( line = dict( color = INCREASING_COLOR ) ),
    decreasing = dict( line = dict( color = DECREASING_COLOR ) ),
) ]

#layout=dict()

fig = dict( data=data, layout=layout )

fig['layout'] = dict()
fig['layout']['plot_bgcolor'] = 'rgb(250, 250, 250)'
fig['layout']['xaxis'] = dict( rangeselector = dict( visible = True ) )
fig['layout']['yaxis'] = dict( domain = [0, 0.2], showticklabels = False )
fig['layout']['yaxis2'] = dict( domain = [0.2, 0.8] )
fig['layout']['legend'] = dict( orientation = 'h', y=0.9, x=0.3, yanchor='bottom' )
fig['layout']['margin'] = dict( t=40, b=40, r=40, l=40 )

#rangeselector=dict(
#    visibe = True,
#    x = 0, y = 0.9,
#    bgcolor = 'rgba(150, 200, 250, 0.4)',
#    font = dict( size = 13 ),
#    buttons=list([
#        dict(count=1,
#             label='reset',
#             step='all'),
#        dict(count=1,
#             label='1yr',
#             step='year',
#             stepmode='backward'),
#        dict(count=3,
#            label='3 mo',
#            step='month',
#            stepmode='backward'),
#        dict(count=1,
#            label='1 mo',
#            step='month',
#            stepmode='backward'),
#        dict(step='all')
#    ]))
#fig['layout']['xaxis']['rangeselector'] = rangeselector

#colors = []
#for i in range(len(zec_sub["close"])):
#    if i != 0:
#        if zec_sub["close"][i] > zec_sub["close"][i-1]:
#            colors.append(INCREASING_COLOR)
#        else:
#            colors.append(DECREASING_COLOR)
#    else:
#        colors.append(DECREASING_COLOR)

fig['data'].append( dict( x=zec_sub.index, y=zec_sub["volume"],                         
                         marker=dict( color=colors ),
                         type='bar', yaxis='y', name='Volume' ) )

py.iplot( fig, filename = 'candlestick-test-3', validate = False )
